In [1]:
# load the library
import numpy as np
import pandas as pd
from tqdm import tqdm

# load selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip
from IPython.display import clear_output
import sqlite3
from pandas.io import sql
import datetime

In [2]:
# handling page refresh
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [3]:
# Set name of your SQLite database
db_name = "freight"

sqlite_db = db_name + '.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [4]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth',1000)
import time
from sklearn.externals import joblib

In [6]:
freight_lanes = pd.read_excel('freight_lanes.xlsx')

In [7]:
# download chromedriver.exe and save to a local drive locaiton:
location_of_chromedriver = "Driver/chromedriver.exe"
site_url = "https://www.searates.com/reference/portdistance"

In [8]:
# set browser and go to google maps
def init_browser(URL = 'https://www.google.com.hk/maps'):
    browser = webdriver.Chrome(location_of_chromedriver)
    browser.get(URL)
    return browser

In [9]:
rate_site = init_browser(site_url)

In [10]:
if_name = rate_site.find_element_by_tag_name("iframe")
rate_site.switch_to_frame(if_name)

# switch back to main page
# rate_site.switch_to_default_content()

In [11]:
def page_wait(look_for, max_wait_time = 20):
    success = False
    counter = 0
    while success == False:
        counter = counter + 1
        time.sleep(1)
        tmp_texts = rate_site.find_elements_by_class_name(look_for)
        for i in range(len(tmp_texts)):
            try:
                tmp_text = tmp_texts[i].text
                if tmp_text != '':
                    success = True
            except:
                pass
        if counter > max_wait_time:
            success = True
    return counter

In [12]:
def handle_output(output_text):
    tmp = output_text
    dist = tmp[:tmp.find('\n')]
    times = tmp[tmp.find('\n')+1:]


    mile = dist[:dist.find('mi')]
    km = dist[dist.find('mi')+3:]
    mile = mile.strip(" ")
    mile = float(mile)
    km = km.strip('(')
    km = km.strip(')')
    km = km.strip(" ")
    km = km.strip('km')
    km = km.strip(' ')
    km = float(km)
#     print(mile)
#     print(km)

    
    days = times[:times.find('day')]
    days = days.strip(' ')
    days = days.strip('Transit Time: ')
    if days == '':
        days = 1
    else:
        days = float(days)

#     print(days)

    knots = times[times.find('day')+5:]
    knots = knots.strip(' ')
    knots = knots.strip('(')
    knots = knots.strip(')')
    knots = knots.strip('knots')
    knots = knots.strip(' ')
    if knots == '':
        knots = 0
    else:
        knots = float(knots)
#     print(knots)
    
    return mile, km, days, knots

In [25]:
def search_route(orig_port, dest_port, wait_time = 10):
    orig_input = rate_site.find_element_by_id('place_d')
    orig_input.clear()
    orig_input.send_keys(orig_port)
    page_wait('ui-menu-item-wrapper', wait_time)
    orig_input.send_keys(Keys.RETURN)
    dest_input = rate_site.find_element_by_id('place_g')
    dest_input.clear()
    dest_input.send_keys(dest_port)
    page_wait('ui-menu-item-wrapper', wait_time)
    dest_input.send_keys(Keys.RETURN)
    search_btn = rate_site.find_elements_by_id('search_button')
    search_btn[0].click()
    page_wait('m-D', wait_time)
    try:
        place_name = rate_site.find_elements_by_class_name('name')
        orig_name = place_name[0].text
        dest_name = place_name[-1].text
        
        full_trip = []
        for i in range(len(place_name)):
            full_trip.append(place_name[i].text)
        
    except:
        orig_name = 'N/A'
        dest_name = 'N/A'
        full_trip = []
    try:
        inner_charges = rate_site.find_elements_by_class_name('inner-charges')
        charge = []
        for i in inner_charges:
            tmp_charge = i.text
            charge.append(tmp_charge)
        orig_cost = charge[0]
        freight_cost = charge[1]
        dest_cost = charge[2]
    except:
        orig_cost = 'N/A'
        freight_cost = 'N/A'
        dest_cost = 'N/A'        

    try:
        
        details = rate_site.find_elements_by_class_name('events')
        detail = []
        mile = 0
        km = 0
        days = 0
        knots = 0
        for i in range(len(details)):
            tmp_details =  details[i].text
            if tmp_details != '':
                detail.append(tmp_details)
                tmp_mile, tmp_km, tmp_days, tmp_knots = handle_output(tmp_details)
                mile = mile + tmp_mile
                km = km + tmp_km
                days = days + tmp_days
                knots = np.max([knots,tmp_knots])

    except:
        mile = 0
        km = 0
        days = 0
        knots = 0
        detail = []
        
    return orig_name, dest_name, full_trip, orig_cost, freight_cost, dest_cost, mile, km, days, knots, detail

In [26]:
search_route('Leliu, China', 'Hakata, Japan', wait_time = 10)

('N/A', 'N/A', [], 'N/A', 'N/A', 'N/A', 0, 0, 0, 0, [])

In [27]:
# Run for loop to get rates
lanes_df = pd.DataFrame()
for i in tqdm(range(len(freight_lanes))):
# for i in tqdm(range(1)):
    orig_port = freight_lanes['ORIG_PORT'].iloc[i]
    dest_port = freight_lanes['DEST_PORT'].iloc[i]
    orig_name, dest_name, full_trip, orig_cost, freight_cost, dest_cost, mile, km, days, knots, detail = search_route(orig_port, dest_port)
    
    tmp_lanes_df = pd.DataFrame()
    tmp_lanes_df['orig_name'] = [orig_name]
    tmp_lanes_df['dest_name'] = dest_name
    tmp_lanes_df['full_trip'] = str(full_trip)
    tmp_lanes_df['orig_cost'] = orig_cost
    tmp_lanes_df['freight_cost'] = freight_cost
    tmp_lanes_df['dest_cost'] = dest_cost
    tmp_lanes_df['mile'] = mile
    tmp_lanes_df['km'] = km
    tmp_lanes_df['days'] = days
    tmp_lanes_df['max_knots'] = knots
    tmp_lanes_df['detail'] = str(detail)
    now = datetime.datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M")
    tmp_lanes_df['extraction_time'] = now
    
#     lanes_df = lanes_df.append(tmp_lanes_df)
    tmp_lanes_df.to_sql('web_freight_rates', con=conn, index=False, if_exists='append')

# lanes_df
# lanes_df.to_csv('lanes_df.csv', index=False, encoding='utf_8')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:31<00:00, 15.31s/it]


In [28]:
pd.read_sql('select * from web_freight_rates', con = conn)

,orig_name,dest_name,full_trip,orig_cost,freight_cost,dest_cost,mile,km,days,max_knots,detail,extraction_time
0,N/A,N/A,[],N/A,N/A,N/A,0,0,0,0,[],2018-06-24 09:57
1,N/A,N/A,[],N/A,N/A,N/A,0,0,0,0,[],2018-06-24 09:57
2,N/A,N/A,[],N/A,N/A,N/A,0,0,0,0,[],2018-06-24 09:57
3,N/A,N/A,[],N/A,N/A,N/A,0,0,0,0,[],2018-06-24 09:57
4,N/A,N/A,[],N/A,N/A,N/A,0,0,0,0,[],2018-06-24 09:58
5,N/A,N/A,[],N/A,N/A,N/A,0,0,0,0,[],2018-06-24 09:58


In [18]:
web_freight_rates = pd.read_sql('select * from web_freight_rates', con = conn)
web_freight_rates.drop_duplicates(inplace=True)
web_freight_rates.to_csv('lanes_time.csv', index=False, encoding='utf_8')

In [19]:
rate_site.close()

In [20]:
conn.close()